Task 2

In [1]:
from pyspark import SparkContext, SparkConf
from functools import reduce
from collections import Counter
import pandas as pd

def load_dataset(filename, columnNames):
    df = pd.read_csv(filename,encoding='latin1',names=columnNames)
    return df

def get_instances(data):
    instances = data.split()
    return Counter(instances)

def ReduceCounter(counter1, counter2):
    counter1.update(counter2)
    return counter1

conf = SparkConf().setAppName('MapReduce').setMaster('local')
sparkContext = SparkContext.getOrCreate(conf=conf)

columns = ['passengerID', 'flightID', 'originAirport', 'destinationAirport', 'departureTime', 'flightTime']

rdd = load_dataset("AComp_Passenger_data_no_error.csv", columns)

Origins = rdd['originAirport'].tolist()
distributed_data_origins = sparkContext.parallelize(Origins, 10)

dist_data_flight_origins = distributed_data_origins.map(get_instances)
dist_data_flight_origins_count = dist_data_flight_origins.reduce(ReduceCounter)
print(dist_data_flight_origins_count)


24/12/03 14:25:13 WARN Utils: Your hostname, codespaces-8bb66a resolves to a loopback address: 127.0.0.1; using 10.0.11.17 instead (on interface eth0)
24/12/03 14:25:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 14:25:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Counter({'DEN': 46, 'CAN': 37, 'IAH': 37, 'ATL': 36, 'ORD': 33, 'KUL': 33, 'CGK': 27, 'JFK': 25, 'LHR': 25, 'CDG': 21, 'CLT': 21, 'PVG': 20, 'LAS': 17, 'BKK': 17, 'AMS': 15, 'FCO': 15, 'MUC': 14, 'MAD': 13, 'PEK': 13, 'HND': 13, 'DFW': 11, 'MIA': 11})


In [4]:
#Flight ID, Count of Passengers, Departure code, Departure Time, Arrival Code, Arrival Time
def GetFlightInfo(IDRecords, table):
    data = pd.DataFrame({'flightID','originAirport','departureTime','destinationAirport','flightTime'})
    for i in range(len(IDRecords)):
        first_instance = (table['flightID'] == IDRecords[i]).idxmax() #https://www.w3schools.com/python/pandas/ref_df_idxmax.asp
        row = table.iloc[first_instance].loc[['flightID','originAirport','departureTime','destinationAirport','flightTime']] #https://www.kdnuggets.com/2019/06/select-rows-columns-pandas.html
        data = pd.concat([data, row]) #https://pandas.pydata.org/docs/reference/api/pandas.concat.html
    return data

def JoinDataFrames(df1, df2):
    pd.merge(df1, df2, on='flightID', how='inner')
    return df1

Flights = rdd['flightID'].tolist()
distributed_data_flights = sparkContext.parallelize(Flights, 10)

dist_data_flight_id = distributed_data_flights.map(get_instances)
dist_data_flight_id_count = dist_data_flight_id.reduce(ReduceCounter)

dist_data_flight_id_list = list(dist_data_flight_id_count.keys())

FlightInfoData = GetFlightInfo(dist_data_flight_id_list, rdd)
FlightInfo = pd.DataFrame(data=FlightInfoData)

FlightDataList = {'flightID': dist_data_flight_id_count.keys(), 'passengerCount': dist_data_flight_id_count.values()}
FlightData = pd.DataFrame(data=FlightDataList)

df = JoinDataFrames(FlightData, FlightInfo)


KeyError: 'flightID'